### FCN Black-box Classifier

In [1]:
import tensorflow as tf
from tensorflow import keras
import time
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, GlobalAveragePooling1D, BatchNormalization, Conv2D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import function
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import sklearn
from scipy.optimize import minimize
from scipy.spatial.distance import cdist, pdist
from scipy import stats
from sklearn.neighbors import DistanceMetric
from tslearn.datasets import UCR_UEA_datasets
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from counterfactual_utils import label_encoder, ucr_data_loader
print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False
%matplotlib inline


TF version:  2.4.1
Eager execution enabled:  False
TF version:  2.4.1
Eager execution enabled:  False


#### Classifier Architecture

This architecture was originally proposed by Wang et al and the implementation closely follows the code provided by Fawaz et al in a fantastic review paper on DNNs for Time series classification.

In [2]:
class Classifier_FCN:
    
    def __init__(self, output_directory, input_shape, nb_classes, dataset_name, verbose=False,build=True):
        self.output_directory = output_directory

        
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if(verbose==True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(str(dataset_name) +'_model_init.hdf5')
        return

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
        conv1 = keras.layers.BatchNormalization()(conv1)
        conv1 = keras.layers.Activation(activation='relu')(conv1)

        conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
        conv2 = keras.layers.BatchNormalization()(conv2)
        conv2 = keras.layers.Activation('relu')(conv2)

        conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
        conv3 = keras.layers.BatchNormalization()(conv3)
        conv3 = keras.layers.Activation('relu')(conv3)

        gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
            metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, 
            min_lr=0.0001)

        file_path = str(dataset_name) +'_best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', 
            save_best_only=True)

        self.callbacks = [reduce_lr,model_checkpoint]

        return model 

    def fit(self, x_train, y_train):
         
        batch_size = 16
        nb_epochs = 2000

        mini_batch_size = int(min(x_train.shape[0]/10, batch_size))
        

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
            verbose=self.verbose, callbacks=self.callbacks)

        self.model.save(str(dataset_name) +'_last_model.hdf5')

        model = keras.models.load_model(str(dataset_name) +'_best_model.hdf5')


    def predict(self, x_test):
        model_path = str(dataset_name) + '_best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred, axis=1)
        return y_pred


#### Training and saving weights 

In [3]:
#for dataset in ['ecg200', 'gunpoint', 'coffee', 'chinatown', 'CBF']:

#    X_train, y_train, X_test, y_test = ucr_data_loader(str(dataset))
#    y_train, y_test = label_encoder(y_train, y_test)
    
#    input_shape = X_train.shape[1:]
#    nb_classes = len(np.unique(np.concatenate([y_train,y_test])))
#    one_hot = to_categorical(y_train)
#    dataset_name = str(dataset)
    
    
#    fcn = Classifier_FCN(output_directory=os.getcwd(), input_shape=input_shape, nb_classes=nb_classes, dataset_name=dataset_name)
#    fcn.build_model(input_shape=input_shape, nb_classes=nb_classes)
#    fcn.fit(X_train, to_categorical(y_train))
#    fcn.predict(X_test)